In [62]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from hyperopt.pyll import scope
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np, xgboost as xgb
from xgboost import  XGBRegressor
from sklearn.model_selection import KFold, train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval


In [55]:
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv', sep=',')
df.head()

,Transaction Hash_len,Original_len,signature_len,From_len,To_len,sender_len,paymaster_len,Txn Fee,Gas Used,logIndex,actualGasCost,actualGasUsed,nonce,success,Blockno,DateTime_ts
0,32,964,4,20,20,20,20,0.022033,398741,245,2.190000e+16,397164,0.0,1,17066994,1681740540
1,32,868,4,20,20,20,20,0.004362,87702,231,4.280000e+15,86113,1.0,1,17067000,1681740600
2,32,868,4,20,20,20,20,0.003971,87714,273,3.900000e+15,86125,2.0,1,17067009,1681740720
3,32,1188,4,20,20,20,20,0.008673,161702,429,8.650000e+15,161161,3.0,1,17078992,1681887000
4,32,1188,4,20,20,20,20,0.006445,127502,393,6.420000e+15,126973,4.0,1,17079029,1681887420


In [58]:

features = ['Transaction Hash_len','Original_len','signature_len',
            'From_len','To_len','sender_len','paymaster_len',
            'Txn Fee','logIndex','actualGasCost',
            'actualGasUsed','nonce','success','Blockno','DateTime_ts']

# Tách X và y
X = df.loc[:, features]
y = df['Gas Used']

# 1) Split trước
X_train, X_val, y_train, y_val = train_test_split(
    X, y, random_state=10, train_size=0.8, shuffle=True
)

print("Before scaling:")
print(f"X_train: {X_train.shape} \ny_train: {y_train.shape} "
      f"\nX_val: {X_val.shape}, \ny_val: {y_val.shape}")

# 2) Scale X bằng MinMaxScaler
scaler_X = MinMaxScaler()
X_train_scaled = pd.DataFrame(
    scaler_X.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_val_scaled = pd.DataFrame(
    scaler_X.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)
# 3) Scale y bằng MinMaxScaler
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1,1)).ravel()
y_val_scaled   = scaler_y.transform(y_val.values.reshape(-1,1)).ravel()

print("\nAfter scaling:")
print(f"X_train_scaled: {X_train_scaled.shape} \ny_train_scaled: {y_train_scaled.shape} "
      f"\nX_val_scaled: {X_val_scaled.shape}, \ny_val_scaled: {y_val_scaled.shape}")

# 4) Kiểm tra min/max sau khi scale
print("\nCheck scaling X_train (min/max):")
print(X_train_scaled.min().round(3))
print(X_train_scaled.max().round(3))

print("\nCheck scaling y_train (min/max):")
print(y_train_scaled.min().round(3), y_train_scaled.max().round(3))

Before scaling:
X_train: (87492, 15) 
y_train: (87492,) 
X_val: (21873, 15), 
y_val: (21873,)

After scaling:
X_train_scaled: (87492, 15) 
y_train_scaled: (87492,) 
X_val_scaled: (21873, 15), 
y_val_scaled: (21873,)

Check scaling X_train (min/max):
Transaction Hash_len    0.0
Original_len            0.0
signature_len           0.0
From_len                0.0
To_len                  0.0
sender_len              0.0
paymaster_len           0.0
Txn Fee                 0.0
logIndex                0.0
actualGasCost           0.0
actualGasUsed           0.0
nonce                   0.0
success                 0.0
Blockno                 0.0
DateTime_ts             0.0
dtype: float64
Transaction Hash_len    0.0
Original_len            1.0
signature_len           0.0
From_len                0.0
To_len                  0.0
sender_len              0.0
paymaster_len           0.0
Txn Fee                 1.0
logIndex                1.0
actualGasCost           1.0
actualGasUsed           1.0
nonce  

In [59]:
space = {
    "reg_params": {
        "learning_rate":    hp.uniform("xgb_lr", 0.01, 0.2),
        "max_depth":        scope.int(hp.choice("xgb_md", [2, 5, 10])),
        "n_estimators":     scope.int(hp.quniform("xgb_ne", 2000, 12000, 500)),
        "reg_lambda":       hp.uniform("xgb_rl", 1.0, 10.0),
        "min_child_weight": scope.int(hp.quniform("xgb_mcw", 1, 200, 1)),
        "gamma":            hp.uniform("xgb_g", 0.0, 2.0),
        "subsample":        hp.uniform("xgb_ss", 0.6, 1.0),
        "colsample_bytree": hp.uniform("xgb_cbt", 0.6, 1.0),
        "objective":        "reg:squarederror",
        "eval_metric":      "rmse",
        "random_state":     10,
        "verbosity":        0,
        "n_jobs":           -1,
        "tree_method":      "gpu_hist",
    },
    "fit_params": {}
}

class HPOptimiser:
    def __init__(self, Xtr, Xval, ytr, yval):
        self.Xtr, self.Xval, self.ytr, self.yval = Xtr, Xval, ytr, yval

    def xgb_reg(self, p):
        rp = p["reg_params"].copy()
        for k in ["max_depth","n_estimators","min_child_weight"]: rp[k] = int(rp[k])
        try:
            return self._train_eval(rp)                  # GPU
        except Exception:
            rp["tree_method"] = "hist"                   # Fallback CPU
            return self._train_eval(rp)

    def _train_eval(self, rp):
        params = rp.copy()
        num_round = int(params.pop("n_estimators"))
        # map tham số cho xgb.train
        if "n_jobs" in params: params["nthread"] = int(params.pop("n_jobs"))
        if "random_state" in params: params["seed"] = int(params.pop("random_state"))
        dtr = xgb.DMatrix(self.Xtr, label=self.ytr)
        dvl = xgb.DMatrix(self.Xval, label=self.yval)
        bst = xgb.train(params, dtr, num_boost_round=num_round,
                        evals=[(dvl, "val")], early_stopping_rounds=10, verbose_eval=False)
        ntree = getattr(bst, "best_ntree_limit", None)
        pred = bst.predict(dvl, ntree_limit=ntree) if ntree else bst.predict(dvl, iteration_range=(0, bst.best_iteration+1))
        rmse = float(np.sqrt(mean_squared_error(self.y_val, pred))) if hasattr(self, "y_val") else float(np.sqrt(mean_squared_error(self.yval, pred)))
        return {"loss": rmse, "status": STATUS_OK}

# Run Hyperopt (scaled)
trials = Trials()
optim  = HPOptimiser(X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled)
best_raw = fmin(fn=optim.xgb_reg, space=space, algo=tpe.suggest, max_evals=30, trials=trials)
best = space_eval(space, best_raw)
for k in ["max_depth","n_estimators","min_child_weight"]:
    best["reg_params"][k] = int(best["reg_params"][k])
print("Best RMSE (scaled):", trials.best_trial["result"]["loss"])
print("Best params:", best["reg_params"])

# Train final (scaled) bằng xgb.train
rp = best["reg_params"].copy()
num_round = int(rp.pop("n_estimators"))
if "n_jobs" in rp: rp["nthread"] = int(rp.pop("n_jobs"))
if "random_state" in rp: rp["seed"] = int(rp.pop("random_state"))
try:
    dtr = xgb.DMatrix(X_train_scaled, label=y_train_scaled)
    dvl = xgb.DMatrix(X_val_scaled,   label=y_val_scaled)
    final = xgb.train(rp, dtr, num_boost_round=num_round,
                      evals=[(dvl,"val")], early_stopping_rounds=10, verbose_eval=False)
except Exception:
    rp["tree_method"] = "hist"
    final = xgb.train(rp, dtr, num_boost_round=num_round,
                      evals=[(dvl,"val")], early_stopping_rounds=10, verbose_eval=False)

ntree = getattr(final, "best_ntree_limit", None)
pred  = final.predict(dvl, ntree_limit=ntree) if ntree else final.predict(dvl, iteration_range=(0, final.best_iteration+1))
val_rmse = float(np.sqrt(mean_squared_error(y_val_scaled, pred)))
best_iter = getattr(final, "best_iteration", "N/A")
print("Final RMSE (scaled):", val_rmse)
print("Best iteration:", best_iter)

100%|██████████| 30/30 [00:47<00:00,  1.58s/trial, best loss: 0.009964083934852425]
Best RMSE (scaled): 0.009964083934852425
Best params: {'colsample_bytree': 0.6505004603312615, 'eval_metric': 'rmse', 'gamma': 0.009381485175251371, 'learning_rate': 0.1897894396945371, 'max_depth': 10, 'min_child_weight': 153, 'n_estimators': 3000, 'n_jobs': -1, 'objective': 'reg:squarederror', 'random_state': 10, 'reg_lambda': 3.580522281314385, 'subsample': 0.9858720992281388, 'tree_method': 'gpu_hist', 'verbosity': 0}
Final RMSE (scaled): 0.009964083934852425
Best iteration: 103


In [64]:
dtest = xgb.DMatrix(X_test_scaled)
kf = KFold(n_splits=10, shuffle=True, random_state=10)
oof_preds_scaled  = np.zeros(len(X_train_scaled))
test_preds_scaled = np.zeros(len(X_test_scaled))
metrics_per_fold  = []

for fold,(tr,va) in enumerate(kf.split(X_train_scaled, y_train_scaled),1):
    rp = best["reg_params"].copy()
    for k in ["max_depth","n_estimators","min_child_weight"]: rp[k] = int(rp[k])
    num = int(rp.pop("n_estimators")); rp["nthread"] = int(rp.pop("n_jobs",-1)); rp["seed"] = int(rp.pop("random_state",10))
    dtr = xgb.DMatrix(X_train_scaled.iloc[tr], label=y_train_scaled[tr])
    dva = xgb.DMatrix(X_train_scaled.iloc[va], label=y_train_scaled[va])
    try:
        bst = xgb.train(rp, dtr, num_boost_round=num, evals=[(dva,"val")], early_stopping_rounds=100, verbose_eval=False)
    except Exception:
        rp["tree_method"]="hist"
        bst = xgb.train(rp, dtr, num_boost_round=num, evals=[(dva,"val")], early_stopping_rounds=100, verbose_eval=False)

    nt = getattr(bst,"best_ntree_limit", None)
    val_pred  = bst.predict(dva,  ntree_limit=nt) if nt else bst.predict(dva,  iteration_range=(0, bst.best_iteration+1))
    test_pred = bst.predict(dtest, ntree_limit=nt) if nt else bst.predict(dtest, iteration_range=(0, bst.best_iteration+1))

    oof_preds_scaled[va] += val_pred
    test_preds_scaled    += test_pred / kf.n_splits

    mse  = mean_squared_error(y_train_scaled[va], val_pred)
    fold_metrics = {"fold": fold, "MSE": mse, "RMSE": np.sqrt(mse),
                    "MAE": mean_absolute_error(y_train_scaled[va], val_pred),
                    "R^2": r2_score(y_train_scaled[va], val_pred)}
    print(fold_metrics); metrics_per_fold.append(fold_metrics)

# Tổng hợp
oof_rmse  = np.sqrt(mean_squared_error(y_train_scaled, oof_preds_scaled))
test_rmse = np.sqrt(mean_squared_error(y_test_scaled,  test_preds_scaled))
metrics_df = pd.DataFrame(metrics_per_fold)
display(metrics_df.style.format({"MSE":"{:.6f}","RMSE":"{:.6f}","MAE":"{:.6f}","R^2":"{:.6f}"}))

{'fold': 1, 'MSE': 5.706994569744772e-05, 'RMSE': np.float64(0.0075544652820333826), 'MAE': 0.0020410668578141845, 'R^2': 0.8828329907865933}
{'fold': 2, 'MSE': 8.455091503832812e-05, 'RMSE': np.float64(0.009195157151366588), 'MAE': 0.0020841682956184117, 'R^2': 0.8375102977093705}
{'fold': 3, 'MSE': 6.982814793282412e-05, 'RMSE': np.float64(0.008356323828863033), 'MAE': 0.0020094859108486314, 'R^2': 0.8695303836140514}
{'fold': 4, 'MSE': 3.247860537475264e-05, 'RMSE': np.float64(0.005699000383817555), 'MAE': 0.0019302695740763846, 'R^2': 0.9234494319203742}
{'fold': 5, 'MSE': 0.00016608006519330137, 'RMSE': np.float64(0.012887205484250703), 'MAE': 0.0021599601648010247, 'R^2': 0.74176575873149}
{'fold': 6, 'MSE': 0.00015022306594869228, 'RMSE': np.float64(0.012256551960020904), 'MAE': 0.002211341009825189, 'R^2': 0.7782268666674148}
{'fold': 7, 'MSE': 5.7226206454828043e-05, 'RMSE': np.float64(0.0075648004900874975), 'MAE': 0.0020221877369538737, 'R^2': 0.8819039018434914}
{'fold': 8,

,fold,MSE,RMSE,MAE,R^2
0,1,0.000057,0.007554,0.002041,0.882833
1,2,0.000085,0.009195,0.002084,0.837510
2,3,0.000070,0.008356,0.002009,0.869530
3,4,0.000032,0.005699,0.001930,0.923449
4,5,0.000166,0.012887,0.002160,0.741766
5,6,0.000150,0.012257,0.002211,0.778227
6,7,0.000057,0.007565,0.002022,0.881904
7,8,0.000095,0.009747,0.002118,0.831161
8,9,0.000105,0.010245,0.002176,0.810346
9,10,0.000100,0.010017,0.002163,0.826631
